In [1]:
import requests
from bs4 import BeautifulSoup
import json

class WebScraper:
    def __init__(self, keyword):
        self.keyword = keyword

    def get_jobs(self):
        all_jobs_data = []
        page = 1
        while True:
            #page_url = f"https://www.seek.co.nz/{self.keyword}-jobs/in-All-New-Zealand?sortmode=ListedDate&page={page}"
            page_url = f"https://www.seek.co.nz/{self.keyword}-jobs/in-All-Auckland?sortmode=ListedDate&page={page}"
            page_data = self.scrape_page(page_url)
            if not page_data:
                break  
            all_jobs_data.extend(page_data)
            page += 1

        jobs_json = json.dumps(all_jobs_data, indent=4)
        print(jobs_json)
        return jobs_json

    def scrape_page(self, url):
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            jobs_data = []

            job_listings = soup.find_all('article', attrs={'data-card-type': 'JobCard'})
            for job in job_listings:
                link_element = job.find('a', {'data-automation': 'jobTitle'})
                job_link = 'https://www.seek.co.nz' + link_element['href'] if link_element else None

                job_data = {
                    'job_id': job.get('data-job-id', None),
                    'job_link': job_link,
                    'title': self.get_text(job, 'a', {'data-automation': 'jobTitle'}),
                    'company': self.get_text(job, 'a', {'data-automation': 'jobCompany'}),
                    'location': self.get_text(job, 'a', {'data-automation': 'jobLocation'}),
                    'short_description': self.get_text(job, 'span', {'data-automation': 'jobShortDescription'})
                }
                jobs_data.append(job_data)
            
            return jobs_data
        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return None
        
    def get_text(self, parent, tag, attrs):
        element = parent.find(tag, attrs)
        return element.get_text(strip=True) if element else None

# using keywords listing
keywords = ["data-engineer", "BI-developer", "data-consultant"]

for keyword in keywords:
    scraper = WebScraper(keyword)
    json_data = scraper.get_jobs()


[
    {
        "job_id": "71790400",
        "job_link": "https://www.seek.co.nz/job/71790400?type=promoted&ref=search-standalone",
        "title": "Senior Cloud Infrastructure Consultant - Azure",
        "company": "Theta",
        "location": "Auckland CBD",
        "short_description": "Great opportunity for Senior Cloud Consultant specialising in Microsoft Azure cloud platform to join the dynamic team in our Digital practice."
    },
    {
        "job_id": "72051767",
        "job_link": "https://www.seek.co.nz/job/72051767?type=promoted&ref=search-standalone",
        "title": "Cloud Systems Engineer",
        "company": "Fisher & Paykel Healthcare",
        "location": "Manukau & East Auckland",
        "short_description": "Do you want to be part of a growing cloud team that works with products that help support patients and clincians around the world? Join F&P"
    },
    {
        "job_id": "72168731",
        "job_link": "https://www.seek.co.nz/job/72168731?type=standout&